<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Training/3_Training_4_ORPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune with ORPO
In this notebook we fine-tune the baseline model with ORPO training
- **Baseline Model**: meta-llama/Llama-3.1-8B-Instruct

**Prerequisits**
- GPU A100 for at least 1 hour
- Access to out GitHub repository
- HF account with registration for Meta Llama 3.1 model family

In [ ]:
!pip install -qqq -U transformers datasets accelerate peft trl bitsandbytes wandb --progress-bar off

In [ ]:
import gc
import os
import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
import logging

### Colab
This part is only relevant when using the notebook in google colab

In [ ]:
from google.colab import userdata, drive

In [ ]:
drive.mount('/content/drive')
token = userdata.get('GITHUB')

### GitHub
Clone GitHub Repository to directly push generated files

In [ ]:
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

### Static Variables

In [ ]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 4
BASE_MODEL_REPO = "meta-llama/Llama-3.1-8B-Instruct"
MODEL_NAME = "Meta-Llama-3.1-8B-Instruct_ORPO"

PUSH_TO_GITHUB = False
PUSH_TO_HF = False

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/NLP2025_CQG/Data/Processed/CQ DPO Dataset.json"

log_base_path = f"/content/NLP2025_CQG/Training/Logs/Traing_{TRAINING_NUMBER}/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/NLP2025_CQG/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)


checkpoint_dir = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)


################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [ ]:
logger.info("--------  Start with ORPO Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

## Training configuration

In [ ]:
# Defined in the secrets tab in Google Colab
wb_token = userdata.get('wandb')
wandb.login(key=wb_token)
torch_dtype = torch.bfloat16

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_REPO)
tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_REPO,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

In [ ]:
dataset = load_dataset('json', data_files=train_dataset_path)
dataset = dataset.filter(lambda x: x['score_chosen'] - x['score_rejected'] > 4)

In [ ]:
print(dataset)

In [ ]:
def format_chat_template(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    row["prompt"] = tokenizer.apply_chat_template(row["prompt"], tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)

In [ ]:
print(dataset)

In [ ]:
columns_to_remove = ['id', 'score_chosen', 'score_rejected', 'schema', 'context']

dataset = dataset['train'].remove_columns(columns_to_remove)

In [ ]:
dataset = dataset.train_test_split(test_size=0.05)

In [ ]:
print(dataset)

In [ ]:
print("---------------   Prompt -----------------")
print(dataset['train'][0]['prompt'])
print("------------------------------------------")
print("---------------   Chosen -----------------")
print("------------------------------------------")
print(dataset['train'][0]['chosen'])
print("------------------------------------------")
print("---------------   Rejected -----------------")
print("------------------------------------------")
print(dataset['train'][0]['rejected'])

## Start training

In [ ]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    beta=0.1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    remove_unused_columns=False,
    optim="paged_adamw_8bit",
    max_steps = 40,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir="./results/",
)

trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    processing_class=tokenizer,
)
trainer.train()
trainer.save_model(MODEL_NAME)

In [ ]:
# Flush memory
#del trainer, model
gc.collect()
gc.collect()
torch.cuda.empty_cache()

# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_REPO)

fp16_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_REPO,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, MODEL_NAME)
model = model.merge_and_unload()

In [ ]:
if PUSH_TO_HF:
    model.push_to_hub(MODEL_NAME, use_temp_dir=False)
    tokenizer.push_to_hub(MODEL_NAME, use_temp_dir=False)

## Push to GitHub

In [ ]:
if PUSH_TO_GITHUB:
    os.chdir("NLP2025_CQG")
    !ls

    !git config --global user.name "Rico Städeli"
    !git config --global user.email "rico@yabriga.ch"

    commit_message = f"Training Number: {TRAINING_NUMBER}, Training logs in Google Drive."
    !git add .
    !git commit -m "{commit_message}"
    !git push